In [1]:
import config
import os, numpy as np, pandas, sklearn
import mido
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
## NN libs
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
import keras
from keras import regularizers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.layers import Input, Dense, Activation, Conv1D, Conv2D, Dropout, Flatten
from keras.layers import Conv2DTranspose, Reshape, MaxPooling2D, UpSampling2D, UpSampling1D, MaxPooling1D
from keras.layers import LocallyConnected1D, LocallyConnected2D
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model
from keras.callbacks import TensorBoard

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
# local libs
import config, models
from data import data, midi
from utils import io, models_io

In [4]:
n: int = 8
context, x_train, labels = data.init(n)
print(x_train.shape)

Setting up params

 >> Context(max_t=10.0, dt=0.01, n_instances=1000, note_length=0.03, bpm=120.0, tempo=500000, ticks_per_beat=480)
Importing midi-data

[INFO] :
 |  reading file: ../datasets/examples/01 16th Snare copy.mid
[INFO] :
 |  reading file: ../datasets/examples/01 16th Snare.mid
[INFO] :
 |  reading file: ../datasets/examples/01 8th Cym copy.mid
[INFO] :
 |  reading file: ../datasets/examples/01 8th Cym.mid
[INFO] :
 |  reading file: ../datasets/examples/01 8th Hat copy.mid
[INFO] :
 |  reading file: ../datasets/examples/01 8th Hat.mid
[INFO] :
 |  reading file: ../datasets/examples/02 8th Cym copy.mid
[INFO] :
 |  reading file: ../datasets/examples/02 8th Cym.mid

Encoding midi-data
 [<midi file '../datasets/examples/01 16th Snare copy.mid' type 0, 1 tracks, 182 messages>, <midi file '../datasets/examples/01 16th Snare.mid' type 0, 1 tracks, 182 messages>, <midi file '../datasets/examples/01 8th Cym copy.mid' type 0, 1 tracks, 68 messages>, <midi file '../datasets/examples/

In [5]:
model = models_io.load_model(config.model_dir + 'model')
encoder = models_io.load_model(config.model_dir + 'encoder')
decoder = models_io.load_model(config.model_dir + 'decoder')

[INFO] :
 |  Loaded model from disk
[INFO] :
 |  compiled model
[INFO] :
 |  Loaded model from disk
[INFO] :
 |  compiled model
[INFO] :
 |  Loaded model from disk
[INFO] :
 |  compiled model


In [7]:
result = encoder.predict(x_train)
n = result.shape[0]
result.shape

(8, 100)

Combine outputs of the encoder

In [8]:
originals = result[0:2,]
new = np.stack([originals.mean(axis=0), originals[0]])

In [10]:
arr = decoder.predict(new)
arr.shape

(2, 1000, 127)

In [11]:
midis = [midi.decode_track(context, track) for track in arr]

[INFO] :
 |  len
 |  10.437499999999952
 |  10.0
[INFO] :
 |  len
 |  10.437499999999952
 |  10.0


In [12]:
midis[0].tracks[0]

<midi track '' 15400 messages>

In [13]:
for i, mid in enumerate(midis):
    io.export_midifile(mid, '../midis/midi_%i.mid' % i)